In [1]:
!nvidia-smi

Wed Apr 27 14:02:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/Softcatala/ca-text-corpus.git

Cloning into 'ca-text-corpus'...
remote: Enumerating objects: 636, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 636 (delta 2), reused 1 (delta 0), pack-reused 629
Receiving objects: 100% (636/636), 33.91 MiB | 11.89 MiB/s, done.
Resolving deltas: 100% (338/338), done.


In [3]:
import shutil

with open('output_file.txt','wb') as wfd:
    for f in ['/content/ca-text-corpus/data/common-voice-sentences.txt',
              '/content/ca-text-corpus/data/dogc.txt',
              '/content/ca-text-corpus/data/dogv.txt',
              '/content/ca-text-corpus/data/riuraueditors.txt',
              '/content/ca-text-corpus/data/softcatala.txt',
              '/content/ca-text-corpus/data/wiki.ca.txt',
              '/content/ca-text-corpus/data/wiki.ca-mozilla_script.txt']:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)

In [4]:
cd /content

/content


In [5]:
import string
import random
# my_file = open("/content/text.txt", "r")

# reading the file
# data = my_file.read()

# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
# data_into_list = data.replace('\n', ' ').split(".")


data_into_list=[line.strip() for line in open('/content/output_file.txt')]
# data_into_list=[line.strip() for line in open('/content/text.txt')]


text_train = open("text_train.txt", "a")  # append modea
labels_train = open("labels_train.txt", "a")  # append modea

text_dev = open("text_dev.txt", "a")  # append modea
labels_dev = open("labels_dev.txt", "a")  # append modea


for j in data_into_list:
  if len(j)< 100:
    label = ""
    text = ""
    for i in j.split(" "):
      try:
          if i[-1] in string.punctuation and i[0].isupper():
            label = label + f"{i[-1]}U "
            text = text + f"{i[:-1].lower()} "

          elif (i[-1] not in string.punctuation and i[0].isupper()):
            label+="OU " 
            text = text + f"{i.lower()} " 

          elif (i[-1] in string.punctuation and i[0].islower()):  
            label+=f"{i[-1]}O "
            text = text + f"{i[:-1].lower()} "

          elif (i[-1] not in string.punctuation and i[0].islower()):  
            label+="OO "
            text = text + f"{i.lower()} "
      except:
        pass
    if len(text.split())== len(label.split()) and len(text)>0:
        if random.random() < .15:       
            text_dev.write(text+"\n")
            labels_dev.write(label+"\n")  
        else:
            text_train.write(text+"\n")
            labels_train.write(label+"\n")   
    else:
      pass                
    # print(label, "\n", j,"\n", text, "\n\n")       
text_dev.close()
labels_dev.close()
text_train.close()
labels_train.close()

In [6]:
pwd

'/content'

In [7]:
mkdir data

In [8]:
!mv /content/labels_dev.txt    /content/data
!mv /content/labels_train.txt  /content/data
!mv /content/text_dev.txt      /content/data
!mv /content/text_train.txt    /content/data

In [9]:
# let's take a look at the data 
print('Text:')
! head -n 20 /content/data/text_train.txt

print('\nLabels:')
! head -n 20 /content/data/labels_train.txt

Text:
bona setmana 
a quina hora comença el concert 
no tenim res per sopar 
a veure si quedem un dia per dinar 
fa molt de temps que no ens veiem 
quina calor 
no sabem per què avui no ha vingut a treballar 
engega el ventilador 
aquest partit no s'acaba mai 
quedem a les vuit a l'estació 
si acabo d'hora aniré a mirar roba 
necessitem vacances 
la setmana que ve anem a viure al pis nou 
volia fer molta feina i entre una cosa i l'altra no vaig fer res de bo 
sempre vol tenir raó 
ets una mica sòmines però em fas riure 
m'agrada quan fas el pi pont a la gespa 
és un cas anem a la platja i es deixa el banyador 
t'agraden totes 
la raquel porta una brusa de niló 

Labels:
OU .O 
OU OO OO OO OO ?O 
OU OO OO OO .O 
OU OO OO OO OO OO OO .O 
OU OO OO OO OO OO OO .O 
OU .O 
OU OO OO OO OO OO OO OO OO .O 
OU OO .O 
OU OO OO OO .O 
OU OO OO OO OO .O 
OU OO ,O OO OO OO .O 
OU .O 
OU OO OO OO OO OO OO OO OO .O 
OU OO OO OO OO OO OO OO OO OO OO OO OO OO OO .O 
OU OO OO .O 
OU OO OO ,O OO OO OO .O 

In [10]:
%%capture

BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [11]:
%%capture
#can take up to 10 minutes

!git clone https://github.com/NVIDIA/apex
%cd apex

!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" --global-option="--fast_layer_norm" ./

In [12]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[NeMo W 2022-04-27 14:05:34 nemo_logging:349] /content/apex/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
      warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
    
[NeMo W 2022-04-27 14:05:39 experimental:28] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.


## Download the data

To prepare the data please refer to the Habr article (in Russian) or to the official NeMo's documentation.

Adopted Python scripts can be found in the averkij's github repo.

We'll download the previously prepared training data.

In [13]:
DATA_DIR = "/content/data"
WORK_DIR = "/content/work"

In [14]:
! ls -l $DATA_DIR

total 93052
-rw-r--r-- 1 root root  4775882 Apr 27 14:02 labels_dev.txt
-rw-r--r-- 1 root root 27056175 Apr 27 14:02 labels_train.txt
-rw-r--r-- 1 root root  9518613 Apr 27 14:02 text_dev.txt
-rw-r--r-- 1 root root 53930258 Apr 27 14:02 text_train.txt


In [15]:
# let's take a look at the data 
print('Text:')
! head -n 10 $DATA_DIR/text_train.txt

print('\nLabels:')
! head -n 10 $DATA_DIR/labels_train.txt

Text:
bona setmana 
a quina hora comença el concert 
no tenim res per sopar 
a veure si quedem un dia per dinar 
fa molt de temps que no ens veiem 
quina calor 
no sabem per què avui no ha vingut a treballar 
engega el ventilador 
aquest partit no s'acaba mai 
quedem a les vuit a l'estació 

Labels:
OU .O 
OU OO OO OO OO ?O 
OU OO OO OO .O 
OU OO OO OO OO OO OO .O 
OU OO OO OO OO OO OO .O 
OU .O 
OU OO OO OO OO OO OO OO OO .O 
OU OO .O 
OU OO OO OO .O 
OU OO OO OO OO .O 


# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model: 
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

See [docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model) for full config description.

In [16]:
MODEL_CONFIG = "punctuation_capitalization_config.yaml"
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 200000

config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [17]:
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item=DATA_DIR

del config.model.test_ds

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [18]:
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.strategy = 'dp'

trainer = pl.Trainer(**config.trainer)

[NeMo W 2022-04-27 14:06:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:456: UserWarning: The flag `devices=1` will be ignored, instead the device specific number 1 will be used
      f"The flag `devices={devices}` will be ignored, "
    
Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-04-27 14:06:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:53: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
      "Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7."
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the traini

# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [19]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)

[NeMo I 2022-04-27 14:06:25 exp_manager:287] Experiments will be logged at /content/apex/nemo_experiments/Punctuation_and_Capitalization/2022-04-27_14-06-25
[NeMo I 2022-04-27 14:06:25 exp_manager:661] TensorboardLogger has been set up


[NeMo W 2022-04-27 14:06:25 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2302: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-04-27 14:06:25 exp_manager:896] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model.

In [20]:
print(1)
print(nemo_nlp.modules.get_pretrained_lm_models_list())

#change for the appropriate model
PRETRAINED_BERT_MODEL = "bert-base-multilingual-uncased"

1
['bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-base-uncased-distilled-squad', 'distilbert-base-cased', 'distilbert-base-cased-distilled-squad', 'distilbert-base-german-cased', 'distilbert-base-multilingual-cased', 'distilbert-ba

In [21]:
NUM_SAMPLES = 200000

config.trainer.max_epochs = 1
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = 10000

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [22]:
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

[NeMo I 2022-04-27 14:06:45 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-multilingual-uncased, vocab_file: None, merges_files: None, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-04-27 14:06:55 modelPT:216] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


...

Tokenization: 100%|██████████| 10.0k/10.0k [00:11<00:00, 864query/s]


[NeMo I 2022-04-27 14:11:19 punctuation_capitalization_dataset:598] Finished initial tokenization.
[NeMo I 2022-04-27 14:11:19 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-04-27 14:11:19 data_preprocessing:410] Min: 4 |                  Max: 36 |                  Mean: 17.9795 |                  Median: 18.0
[NeMo I 2022-04-27 14:11:19 data_preprocessing:412] 75 percentile: 22.00
[NeMo I 2022-04-27 14:11:19 data_preprocessing:413] 99 percentile: 29.00
[NeMo I 2022-04-27 14:11:19 punctuation_capitalization_dataset:600] Finished clipping and padding.
[NeMo I 2022-04-27 14:11:19 punctuation_capitalization_dataset:602] *** Example ***
[NeMo I 2022-04-27 14:11:19 punctuation_capitalization_dataset:603] i: 0
[NeMo I 2022-04-27 14:11:19 punctuation_capitalization_dataset:604] subtokens: 101 143 10106 11191 10210 11273 10126 10188 10112 16360 23076 10112 102
[NeMo I 2022-04-27 14:11:19 punctuation_capitalization_dataset:605] subtokens_mask: False True True T

Batch building: 100%|██████████| 218/218 [00:00<00:00, 885.92batch/s]


Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertEncoder: ['cls.seq_relationship.bia

In [23]:
!nvidia-smi

Wed Apr 27 14:11:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    30W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2022-04-27 14:11:58 modelPT:576] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        eps: 1e-08
        lr: 2e-05
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2022-04-27 14:11:58 lr_scheduler:837] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7f155294ea90>" 
    will be used during training (effective maximum steps = 12942) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 12942
    )



  | Name             | Type             | Params
------------------------------------------------------
0 | metrics          | ModuleDict       | 0     
1 | bert_model       | BertEncoder      | 167 M 
2 | punct_classifier | TokenClassifier  | 12.3 K
3 | capit_classifier | TokenClassifier  | 1.5 K 
4 | loss             | CrossEntropyLoss | 0     
5 | agg_loss         | AggregatorLoss   | 0     
------------------------------------------------------
167 M     Trainable params
0         Non-trainable params
167 M     Total params
334.741   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2022-04-27 14:11:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


...

[NeMo I 2022-04-27 17:28:59 punctuation_capitalization_model:332] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         98.63      99.10      98.86      98943
    ! (label_id: 1)                                          0.00       0.00       0.00         47
    " (label_id: 2)                                         52.17      34.29      41.38         35
    # (label_id: 3)                                          0.00       0.00       0.00          0
    ' (label_id: 4)                                          0.00       0.00       0.00          1
    * (label_id: 5)                                          0.00       0.00       0.00          0
    + (label_id: 6)                                          0.00       0.00       0.00          0
    , (label_id: 7)                                         74.77      68.36      71.42       4106
    - (label_i


Batch mark up: 100%|██████████| 200000/200000 [00:00<00:00, 517506.72query/s]

Batch building: 100%|██████████| 4314/4314 [00:06<00:00, 667.30batch/s]
Epoch 1, global step 8628: 'val_loss' reached 0.09121 (best 0.09121), saving model to '/content/apex/nemo_experiments/Punctuation_and_Capitalization/2022-04-27_14-06-25/checkpoints/Punctuation_and_Capitalization--val_loss=0.0912-epoch=1.ckpt' as top 3


Validation: 0it [00:00, ?it/s]

[NeMo I 2022-04-27 19:08:06 punctuation_capitalization_model:332] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         98.61      99.16      98.88      98943
    ! (label_id: 1)                                          0.00       0.00       0.00         47
    " (label_id: 2)                                         52.17      34.29      41.38         35
    # (label_id: 3)                                          0.00       0.00       0.00          0
    ' (label_id: 4)                                          0.00       0.00       0.00          1
    * (label_id: 5)                                          0.00       0.00       0.00          0
    + (label_id: 6)                                          0.00       0.00       0.00          0
    , (label_id: 7)                                         75.89      67.78      71.61       4106
    - (label_i


Batch mark up: 100%|██████████| 200000/200000 [00:00<00:00, 531128.94query/s]

Batch building: 100%|██████████| 4314/4314 [00:05<00:00, 752.88batch/s] 
Epoch 2, global step 12942: 'val_loss' reached 0.09210 (best 0.09121), saving model to '/content/apex/nemo_experiments/Punctuation_and_Capitalization/2022-04-27_14-06-25/checkpoints/Punctuation_and_Capitalization--val_loss=0.0921-epoch=2.ckpt' as top 3


In [32]:
queries = [
        "si acabo d'hora aniré a mirar roba",
        "necessitem vacances",
        "a partir d'aquí",
        "acabat el debat procedirem a la votació",
        "ah déu meu",
        "bona tarda diputats diputades",
        "a barcelona i a cubells deu mules són cinc parells",
        "a beure i a menjar mesura has de posar",
        "a bon començament bon acabament",
        "a bon mercat ixen diners i a bon lloguer feiners",
        "a bon mercat vendràs doblat",
        "a bon recollidor pare fill escampador",
        "a bona campana bon batall"
    ]

inference_results = model.add_punctuation_capitalization(queries)

for query, result in zip(queries, inference_results):
    print(f'Query   : {query}')
    print(f'Combined: {result.strip()}\n')

[NeMo I 2022-04-27 19:33:33 punctuation_capitalization_model:1057] Using batch size 13 for inference
[NeMo I 2022-04-27 19:33:33 punctuation_capitalization_infer_dataset:91] Max length: 16
[NeMo I 2022-04-27 19:33:33 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-04-27 19:33:33 data_preprocessing:410] Min: 3 |                  Max: 14 |                  Mean: 8.384615384615385 |                  Median: 8.0
[NeMo I 2022-04-27 19:33:33 data_preprocessing:412] 75 percentile: 11.00
[NeMo I 2022-04-27 19:33:33 data_preprocessing:413] 99 percentile: 13.88


100%|██████████| 1/1 [00:00<00:00,  1.36batch/s]

Query   : si acabo d'hora aniré a mirar roba
Combined: Si acabo d'hora, aniré a mirar roba.

Query   : necessitem vacances
Combined: Necessitem vacances.

Query   : a partir d'aquí
Combined: A partir d'aquí.

Query   : acabat el debat procedirem a la votació
Combined: Acabat el debat, procedirem a la votació.

Query   : ah déu meu
Combined: Ah, Déu meu.

Query   : bona tarda diputats diputades
Combined: Bona tarda Diputats diputades.

Query   : a barcelona i a cubells deu mules són cinc parells
Combined: A Barcelona i a Cubells, deu mules són cinc parells.

Query   : a beure i a menjar mesura has de posar
Combined: A beure i a menjar mesura, has de posar.

Query   : a bon començament bon acabament
Combined: A bon començament, bon acabament.

Query   : a bon mercat ixen diners i a bon lloguer feiners
Combined: A bon mercat ixen diners i a bon lloguer feiners.

Query   : a bon mercat vendràs doblat
Combined: A bon mercat vendràs doblat.

Query   : a bon recollidor pare fill escampador
Co

In [ ]:
Si acabo d'hora, aniré a mirar roba.
Necessitem vacances.
A partir d'aquí?
Acabat el debat, procedirem a la votació.
Ah, Déu meu!
Bona tarda, diputats, diputades.
A Barcelona i a Cubells, deu mules són cinc parells.
A beure i a menjar, mesura has de posar.


A bon començament, bon acabament.
A bon mercat ixen diners i a bon lloguer, feiners.
A bon mercat vendràs doblat.
A bon recollidor pare, fill escampador.
A bona campana, bon batall.

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
!mkdir /content/drive/MyDrive/nemo

In [35]:
%cp -r /content/apex/nemo_experiments/Punctuation_and_Capitalization/2022-04-27_14-06-25/checkpoints/Punctuation_and_Capitalization--val_loss=0.0921-epoch=2-last.ckpt /content/drive/MyDrive/nemo/

In [36]:
%cp -r /content/apex/nemo_experiments/Punctuation_and_Capitalization/2022-04-27_14-06-25/checkpoints/Punctuation_and_Capitalization.nemo /content/drive/MyDrive/nemo/
